In [1]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from C:\Users\noy-4\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\noy-4\AppData\Local\Temp\tmp185e0mvv
  JVM stdout: C:\Users\noy-4\AppData\Local\Temp\tmp185e0mvv\h2o_noy_4_started_from_python.out
  JVM stderr: C:\Users\noy-4\AppData\Local\Temp\tmp185e0mvv\h2o_noy_4_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,12 secs
H2O cluster timezone:,America/Bogota
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_noy_4_zg2gmw
H2O cluster total nodes:,1
H2O cluster free memory:,1.311 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [4]:
data = h2o.import_file('iris.data',col_names=['sepal length','sepal width','petal length','petal width','class'])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
# Input parameters that are going to train
training_columns = ['sepal length','sepal width','petal length','petal width']
# Output parameter train against input parameters
response_column = 'class'

In [6]:
# Split data into train and testing
train, test = data.split_frame(ratios=[0.8])

In [7]:
# Define model
model = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)

In [9]:
# Train model
model.train(x=training_columns, y=response_column, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [11]:
# Model performance
performance = model.model_performance(test_data=test)

print(performance)


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.014032181713438419
RMSE: 0.1184575101605568
LogLoss: 0.058730060121673935
Mean Per-Class Error: 0.03333333333333333
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
11.0,0.0,0.0,0.0,0 / 11
0.0,12.0,0.0,0.0,0 / 12
0.0,1.0,9.0,0.1,1 / 10
11.0,13.0,9.0,0.0303030,1 / 33


Top-3 Hit Ratios: 


k,hit_ratio
1,0.969697
2,1.0
3,1.0


## GBM

In [14]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [15]:
# GBM hyperparameters
gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

In [16]:
# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)

In [17]:
gbm_grid1.train(x=training_columns, y=response_column,
                training_frame=train,
                validation_frame=test,
                ntrees=100,
                seed=1)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [21]:
gbm_gridperf1 = gbm_grid1.get_grid(sort_by="accuracy",decreasing=True)
gbm_gridperf1

     col_sample_rate learn_rate max_depth sample_rate           model_ids  \
0                0.5       0.01         3         0.8   gbm_grid1_model_2   
1                0.5       0.01         9         0.8  gbm_grid1_model_14   
2                1.0        0.1         5         1.0  gbm_grid1_model_30   
3                0.2       0.01         3         1.0  gbm_grid1_model_19   
4                1.0       0.01         9         1.0  gbm_grid1_model_33   
5                0.2       0.01         3         0.8   gbm_grid1_model_1   
6                0.5       0.01         5         1.0  gbm_grid1_model_26   
7                0.2       0.01         9         1.0  gbm_grid1_model_31   
8                0.5       0.01         9         1.0  gbm_grid1_model_32   
9                1.0       0.01         5         1.0  gbm_grid1_model_27   
10               0.5       0.01         3         1.0  gbm_grid1_model_20   
11               1.0       0.01         3         0.8   gbm_grid1_model_3   

In [22]:
# Grab the top GBM model, chosen by validation AUC
best_gbm1 = gbm_gridperf1.models[0]

In [25]:
best_gbm_perf1 = best_gbm1.model_performance(test)
best_gbm_perf1.confusion_matrix()
# 0.7781778619721595

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
11.0,0.0,0.0,0.0,0 / 11
0.0,12.0,0.0,0.0,0 / 12
0.0,0.0,10.0,0.0,0 / 10
11.0,12.0,10.0,0.0,0 / 33


In [13]:
h2o.ls()

,key
0,DRF_model_python_1559841852285_1
1,DRF_model_python_1559841852285_1_cv_1
2,DRF_model_python_1559841852285_1_cv_10
3,DRF_model_python_1559841852285_1_cv_2
4,DRF_model_python_1559841852285_1_cv_3
5,DRF_model_python_1559841852285_1_cv_4
6,DRF_model_python_1559841852285_1_cv_5
7,DRF_model_python_1559841852285_1_cv_6
8,DRF_model_python_1559841852285_1_cv_7
9,DRF_model_python_1559841852285_1_cv_8


In [26]:
type(data)

h2o.frame.H2OFrame

In [27]:
data2 = h2o.import_file("../Dataset/Creditcard/creditcard.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
data2.head()

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.35981,-0.0727812,2.53635,1.37816,-0.338321,0.462388,0.239599,0.0986979,0.363787,0.0907942,-0.5516,-0.617801,-0.99139,-0.311169,1.46818,-0.470401,0.207971,0.0257906,0.403993,0.251412,-0.0183068,0.277838,-0.110474,0.0669281,0.128539,-0.189115,0.133558,-0.0210531,149.62,0
0,1.19186,0.266151,0.16648,0.448154,0.0600176,-0.0823608,-0.078803,0.0851017,-0.255425,-0.166974,1.61273,1.06524,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.0690831,-0.225775,-0.638672,0.101288,-0.339846,0.16717,0.125895,-0.0089831,0.0147242,2.69,0
1,-1.35835,-1.34016,1.77321,0.37978,-0.503198,1.8005,0.791461,0.247676,-1.51465,0.207643,0.624501,0.0660837,0.717293,-0.165946,2.34586,-2.89008,1.10997,-0.121359,-2.26186,0.52498,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.0553528,-0.0597518,378.66,0
1,-0.966272,-0.185226,1.79299,-0.863291,-0.0103089,1.2472,0.237609,0.377436,-1.38702,-0.0549519,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.05965,-0.684093,1.96578,-1.23262,-0.208038,-0.1083,0.0052736,-0.190321,-1.17558,0.647376,-0.221929,0.0627228,0.0614576,123.5,0
2,-1.15823,0.877737,1.54872,0.403034,-0.407193,0.0959215,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.34585,-1.11967,0.175121,-0.451449,-0.237033,-0.0381948,0.803487,0.408542,-0.0094307,0.798278,-0.137458,0.141267,-0.20601,0.502292,0.219422,0.215153,69.99,0
2,-0.425966,0.960523,1.14111,-0.168252,0.420987,-0.0297276,0.476201,0.260314,-0.568671,-0.371407,1.34126,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.0581328,0.0686531,-0.0331938,0.0849677,-0.208254,-0.559825,-0.0263977,-0.371427,-0.232794,0.105915,0.253844,0.0810803,3.67,0
4,1.22966,0.141004,0.0453708,1.20261,0.191881,0.272708,-0.005159,0.0812129,0.46496,-0.0992543,-1.41691,-0.153826,-0.751063,0.167372,0.0501436,-0.443587,0.00282051,-0.611987,-0.045575,-0.219633,-0.167716,-0.27071,-0.154104,-0.780055,0.750137,-0.257237,0.0345074,0.00516777,4.99,0
7,-0.644269,1.41796,1.07438,-0.492199,0.948934,0.428118,1.12063,-3.80786,0.615375,1.24938,-0.619468,0.291474,1.75796,-1.32387,0.686133,-0.076127,-1.22213,-0.358222,0.324505,-0.156742,1.94347,-1.01545,0.0575035,-0.649709,-0.415267,-0.0516343,-1.20692,-1.08534,40.8,0
7,-0.894286,0.286157,-0.113192,-0.271526,2.6696,3.72182,0.370145,0.851084,-0.392048,-0.41043,-0.705117,-0.110452,-0.286254,0.0743554,-0.328783,-0.210077,-0.499768,0.118765,0.570328,0.0527357,-0.0734251,-0.268092,-0.204233,1.01159,0.373205,-0.384157,0.0117474,0.142404,93.2,0
9,-0.338262,1.11959,1.04437,-0.222187,0.499361,-0.246761,0.651583,0.0695386,-0.736727,-0.366846,1.01761,0.83639,1.00684,-0.443523,0.150219,0.739453,-0.54098,0.476677,0.451773,0.203711,-0.246914,-0.633753,-0.120794,-0.38505,-0.069733,0.0941988,0.246219,0.0830756,3.68,0


In [36]:
data2.col_names

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount',
 'Class']

In [39]:
xs =['Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']
ys=['Class']

In [40]:
# Split data into train and testing
# Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each
train,test,valid = prostate_df.split_frame(ratios=[.7, .15])
train2, test2 = data2.split_frame(ratios=[0.7])

In [47]:
estimator = H2ORandomForestEstimator(
    # Stops fitting new trees when 10-tree rolling average is within 0.00001
    stopping_rounds = 10,
    stopping_tolerance = 0.00001,
    stopping_metric = 'auto',
    score_each_iteration = True,
    balance_classes = True,
    seed = 7)

hyper_parameters = {'ntrees':[5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
                    'max_depth':[5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

criteria = {"strategy": "RandomDiscrete", 
            "stopping_rounds": 10,
            "stopping_tolerance": 0.00001,
            "stopping_metric": "misclassification"}

grid_search = H2OGridSearch(model = estimator, 
                            hyper_params = hyper_parameters,
                            search_criteria = criteria)

In [ ]:
grid_search.train(x = xs,
                  y = ys,
                  training_frame = train2,
                  validation_frame = train2)

drf Grid Build progress: |████████████████████████████████████████████████

In [46]:
rf_grid1.train(x=xs, y=ys,
                training_frame=train2,
                validation_frame=test2,
                ntrees=100,
                seed=2)

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: n_estimators of function: grid: Unknown hyper parameter for grid search!
  Request: POST /99/Grid/drf
    data: {'hyper_parameters': "{'n_estimators': [120, 130, 140, 150], 'max_depth': [4, 5, 6, 7, 8]}", 'ntrees': '100', 'seed': '2', 'training_frame': 'py_13_sid_8159', 'validation_frame': 'py_14_sid_8159', 'response_column': 'Class', 'grid_id': 'rf_grid1'}
